In [17]:
import pandas as pd

* The on_bad_lines='skip' parameter ensures that any problematic rows in the CSV files are ignored, preventing errors during loading.
* The sep parameter specifies the delimiter used in each file.

In [18]:
# Load datasets from CSV files with error handling
facebook_df = pd.read_csv('facebook_dataset.csv', on_bad_lines='skip')
google_df = pd.read_csv('google_dataset.csv', sep=',', on_bad_lines='skip')
website_df = pd.read_csv('website_dataset.csv', sep=';', on_bad_lines='skip')


<ipython-input-18-619ac668be91>:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  google_df = pd.read_csv('google_dataset.csv', sep=',', on_bad_lines='skip')


* I've created a new column, company_name, in each DataFrame by normalizing existing name columns. It converts names to lowercase and removes leading/trailing whitespace to ensure consistency, which is good for merging datasets accurately.

In [19]:
# Normalize company names
facebook_df['company_name'] = facebook_df['name'].str.lower().str.strip()
google_df['company_name'] = google_df.get('name', '').str.lower().str.strip()
website_df['company_name'] = website_df['site_name'].str.lower().str.strip()


* I've merged the three DataSets based on the company_name column. The how='outer' parameter ensures that all records from both DataSets are included, even if there’s no match. Suffixes are added to see if there are overlapping columns.

In [20]:
# Merge datasets on company_name
merged_df = facebook_df.merge(google_df, on='company_name', how='outer', suffixes=('_fb', '_google'))
merged_df = merged_df.merge(website_df, on='company_name', how='outer', suffixes=('', '_website'))

print("Merged DataFrame Columns:", merged_df.columns)

Merged DataFrame Columns: Index(['domain_fb', 'address_fb', 'categories', 'city_fb', 'country_code_fb',
       'country_name_fb', 'description', 'email', 'link', 'name_fb',
       'page_type', 'phone_fb', 'phone_country_code_fb', 'region_code_fb',
       'region_name_fb', 'zip_code_fb', 'company_name', 'address_google',
       'category', 'city_google', 'country_code_google', 'country_name_google',
       'name_google', 'phone_google', 'phone_country_code_google',
       'raw_address', 'raw_phone', 'region_code_google', 'region_name_google',
       'text', 'zip_code_google', 'domain_google', 'root_domain',
       'domain_suffix', 'language', 'legal_name', 'main_city', 'main_country',
       'main_region', 'phone', 'site_name', 'tld', 's_category'],
      dtype='object')


* I've made a list of relevant columns to keep in the final merged DataSet. It checks if each desired column exists in the merged DataSet and adds it to the columns_to_keep list. After preparing the list, it filters merged_df to retain only those columns and prints the size of the resulting DataSet.



In [21]:
# Prepare a list of columns to keep, excluding None values
columns_to_keep = ['company_name']

# Add columns conditionally if they exist
if 'categories' in merged_df:
    columns_to_keep.append('categories')
if 'address' in merged_df:
    columns_to_keep.append('address')
if 'raw_address' in merged_df:
    columns_to_keep.append('raw_address')
if 'city' in merged_df:
    columns_to_keep.append('city')
if 'country_name' in merged_df:
    columns_to_keep.append('country_name')
if 'main_country' in merged_df:
    columns_to_keep.append('main_country')
if 'region_name' in merged_df:
    columns_to_keep.append('region_name')
if 'main_region' in merged_df:
    columns_to_keep.append('main_region')
if 'phone' in merged_df:
    columns_to_keep.append('phone')
if 'raw_phone' in merged_df:
    columns_to_keep.append('raw_phone')

# Limit to relevant columns before applying conflict resolution
merged_df = merged_df[columns_to_keep]

print(f"Merged DataFrame Size: {merged_df.shape}")


Merged DataFrame Size: (527738, 7)


* I've tried to resolves conflicts in the merged DataSet by creating a new DataSet, merged_dataset, using the apply function. For each row, it chooses the most relevant values for columns like address, city, and phone, preferring one over the other when there are overlaps. The index is then reset to maintain a clean structure.

In [22]:
# Function to resolve conflicts
merged_dataset = merged_df.apply(lambda row: pd.Series({
    'company_name': row['company_name'],
    'categories': ', '.join(filter(None, [str(row.get('categories', ''))])),
    'address': row.get('address', '') or row.get('raw_address', ''),
    'city': row.get('city', '') or row.get('main_city', ''),
    'country': row.get('country_name', '') or row.get('main_country', ''),
    'region': row.get('region_name', '') or row.get('main_region', ''),
    'phone': row.get('phone', '') or row.get('raw_phone', ''),
}), axis=1)

merged_dataset.reset_index(drop=True, inplace=True) # Reset index


In [23]:
# Save the final dataset and print it
merged_dataset.to_csv('merged_dataset.csv', index=False)

print(merged_dataset)

                                 company_name  \
0                                euro hygiène   
1                      lakeside home services   
2                              rossiter boats   
3       chandler associates architecture inc.   
4                apex surety & insurance ltd.   
...                                       ...   
527733                              pansports   
527734                           turkey point   
527735                      279 bistro urbain   
527736                                   naid   
527737              studio ae interior design   

                                               categories  \
0                                                     nan   
1       Appliance Repair & Maintenance|Home Builders &...   
2             Boats & Yachts Dealers|Boat Tours & Cruises   
3       Architects & Architectural Services|Other Engi...   
4                                                     nan   
...                                          